In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime

## 1. DATA LOADING


In [2]:
# general data including vaccines
patients = pd.read_csv('../data/patients_vaccines.csv', sep = '\t')

In [3]:
# pcr and antigen tests
pcr = pd.read_csv('../data/pcr.csv', sep = '\t')

In [4]:
# serology tests
sero = pd.read_csv('../data/serology.csv', sep = '\t')

In [5]:
# hospitalized patients info
hosp = pd.read_csv('../data/variables_h.csv', sep = '\t')

In [330]:
# Decease date info
dec = pd.read_csv('../data/Dades_ReCOVID_exitus.csv', sep = '\t')

## 2. DATA CLEANING, FORMATTING AND MANAGING

In [6]:
list(patients.columns)

['ID_ANONIMITZAT',
 'NHC_ANONIMITZAT',
 'CIP_ANONIMITZAT',
 'ETIQUETA_1_ANOMIM',
 'DATA PETICIO POS 1',
 'DATA POSITIU 1',
 'SEQ_MOSTRA_1',
 'TIPUS_ETIQ_1',
 'CT_ETIQ_1',
 'ETIQUETA_2_ANOMIM',
 'DATA PETICIO POS 2',
 'DATA POSITIU 2',
 'SEQ_MOSTRA_2',
 'TIPUS_ETIQ_2',
 'CT_ETIQ_2',
 'INGRES_GT',
 'EDAT',
 'SEXE',
 'MACA',
 'PCC',
 'GMA',
 'DATA_D1',
 'VACUNA_D1',
 'DATA_D2',
 'VACUNA_D2',
 'DATA_D3',
 'VACUNA_D3',
 'DATA_D4',
 'VACUNA_D4',
 'INICI_SIMPTOMES_AP']

In [7]:
# renaming columns
patients = patients.rename({'ID_ANONIMITZAT':'id', 'DATA PETICIO POS 1':'pet_date_1', 
                           'DATA POSITIU 1':'pos_date_1', 'SEQ_MOSTRA_1':'seq_1',
                           'TIPUS_ETIQ_1':'type_1', 'CT_ETIQ_1':'ct_1',
                           'DATA PETICIO POS 2':'pet_date_2', 'DATA POSITIU 2':'pos_date_2',
                           'SEQ_MOSTRA_2':'seq_2', 'TIPUS_ETIQ_2':'type_2',
                           'CT_ETIQ_2':'ct_2', 'INGRES_GT':'hosp',
                           'EDAT':'age', 'SEXE':'sex', 'MACA':'maca', 'PCC':'pcc', 'GMA':'gma',
                           'DATA_D1':'v1_date', 'VACUNA_D1':'v1',
                           'DATA_D2':'v2_date', 'VACUNA_D2':'v2',
                           'DATA_D3':'v3_date', 'VACUNA_D3':'v3',
                           'DATA_D4':'v4_date', 'VACUNA_D4':'v4',
                           'INICI_SIMPTOMES_AP':'symp_date'}, axis = 1)

In [8]:
pcr = pcr.rename({'ID_ANONIMITZAT':'id', 'PROVA':'test', 'DATA':'pcr_date', 'RESULTAT':'pcr_result'}, axis = 1)

In [9]:
sero = sero.rename({'ID_ANONIMITZAT':'id', 'DATA_SEROLOGIA':'sero_date', 'RESULTAT':'sero_result'}, axis = 1)

In [10]:
list(hosp.columns)

['ID_ANONIMITZAT',
 'TIPUS_EPISODI',
 'EPISODI_ANONIMITZAT',
 'DATA_INGRES',
 'DATA_ALTA',
 'ESTADES',
 'GRD',
 'GRD_DESC',
 'DIAG_PPAL',
 'DIAG_PPAL_DESC',
 'CDM',
 'EMBARAS',
 'SATO2_VALOR',
 'SATO2_DATA',
 'SATO2_SUPORT',
 'GMA',
 'D_INGRES_UCI',
 'D_ALTA_UCI',
 'D_INGRES_SEMI',
 'D_ALTA_SEMI',
 'VMI',
 'VMNI',
 'CNAF',
 'ECMO',
 'CORTICOIDS',
 'INICI_CORTICOIDS',
 'FI_CORTICOIDS',
 'HEPARINA',
 'INICI_HEPARINA',
 'FI_HEPARINA',
 'EXITUS_A_30_DIAS',
 'REINGRES_30D',
 'GENERA_REINGRES_30D',
 'INMUNOSUPRESSORS_PREVI',
 'INICI_INMUNOSUPRESSORS',
 'FI_INMUNOSUPRESSORS',
 'FAM_DIRECTE_COVID',
 'SIMPTOMATOLOGIA',
 'CEFALEA',
 'DIES_CEFALEA',
 'FEBRE',
 'DIES_FEBRE',
 'DISPNEA_ASTENIA',
 'DIES_DISP_AST',
 'DIARREA',
 'MIALGIES_HIPERALGESIA',
 'DOLOR_TORACIC',
 'ANOSMIA_AGEUSIA',
 'MANIFEST_CUTANIES',
 'DIES_MANIFEST_CUTANIES',
 'ALTRES_SIMPT',
 'DIES_ALTRES_SIMPT',
 'ESPEC_SIMPT',
 'TRACT_ANTIBIOTIC_PREVI',
 'ESPEC_ANTIBIOTIC',
 'RX_PREV_EPISODI',
 'DATA_RX',
 'RX_PATOLOGICA',
 'RX_INI_EPI

In [11]:
hosp = hosp.rename({'ID_ANONIMITZAT':'id', 'DATA_INGRES':'entry_date',
                   'DATA_ALTA':'discharge_date', 'DIAG_PPAL':'diag',
                   'DIAG_PPAL_DESC':'diag_desc', 'D_INGRES_UCI':'uci_e_date',
                   'D_ALTA_UCI':'uci_d_date', 'D_INGRES_SEMI':'ucri_e_date',
                   'D_ALTA_SEMI':'ucri_d_date'}, axis = 1)

In [12]:
# keeping needed variables
# death status is going to be excluded as it doesn't have a linked date
# however it will be useful later
hosp = hosp[['id', 'entry_date', 'discharge_date', 'diag', 'diag_desc', 'uci_e_date', 'uci_d_date',
            'ucri_e_date', 'ucri_d_date']]

In [13]:
# splitting patients dataframe
# tests
test1 = patients[['id', 'pet_date_1', 'pos_date_1', 'seq_1', 'type_1', 'ct_1']]
test2 = patients[['id', 'pet_date_2', 'pos_date_2', 'seq_2', 'type_2', 'ct_2']]
# vaccines
v1 = patients[['id', 'v1_date', 'v1']]
v2 = patients[['id', 'v2_date', 'v2']]
v3 = patients[['id', 'v3_date', 'v3']]
v4 = patients[['id', 'v4_date', 'v4']]
# patients information
info = patients[['id', 'hosp', 'age', 'sex', 'maca', 'pcc', 'gma', 'symp_date']]

In [519]:
# symptoms - for clinical profile
symp = patients[['id', 'symp_date']]

In [521]:
# Codes
pd.read_csv('../data/codes.csv', sep = ',')

,Event,Code
0,PCR/TMA+,A
1,PCR/TMA-,B
2,PCR sequenced,C
3,PCR potential seq.,D
4,Antigen+,E
5,Antigen-,F
6,Serology+,G
7,Serology-,H
8,Pfizer,I
9,Moderna,J


In [15]:
# coding tables
# vaccines
# 1st dose
conditions = [
    (v1['v1'] == 'BioNTech / Pfizer'),
    (v1['v1'] == 'Moderna / Lonza'),
    (v1['v1'] == 'J&J / Janssen'),
    (v1['v1'] == 'Oxford / AstraZeneca')
]
choices = ['I', 'J', 'L', 'K']
v1['event'] = np.select(conditions, choices, default='NaN')
v1 = v1.rename({'v1_date':'date'}, axis = 1)

In [16]:
v1 = v1[['id', 'date', 'event']]
v1 = v1[v1['event'] != 'NaN']

In [17]:
v1['date'] = [str(datetime.strptime(i, '%d/%m/%Y').date()) for i in v1['date']]

In [18]:
v1

,id,date,event
0,fc34ff975cf7159e,2021-01-14,I
1,dd656556a4d03913,2021-04-16,I
2,5d4f2cc96647b02d,2021-06-10,J
3,0efe2e67b10abefb,2021-01-04,I
4,6b0f04b8451798ed,2021-02-16,I
...,...,...,...
3339,fd4611c0467cf512,2021-02-11,J
3341,a27766b3c5d4e85f,2021-04-02,I
3342,f471895734ac5352,2021-05-18,I
3344,310774418891f607,2021-01-15,I


In [19]:
# 2nd dose
conditions = [
    (v2['v2'] == 'BioNTech / Pfizer'),
    (v2['v2'] == 'Moderna / Lonza'),
    (v2['v2'] == 'Oxford / AstraZeneca')
]
choices = ['I', 'J', 'K']
v2['event'] = np.select(conditions, choices, default = 'NaN')
v2 = v2.rename({'v2_date':'date'}, axis = 1)

In [20]:
v2 = v2[['id', 'date', 'event']]
v2 = v2[v2['event'] != 'NaN']

In [21]:
v2['date'] = [str(datetime.strptime(i, '%d/%m/%Y').date()) for i in v2['date']]

In [22]:
# 3rd dose
conditions = [
    (v3['v3'] == 'BioNTech / Pfizer'),
    (v3['v3'] == 'Moderna / Lonza'),
    (v3['v3'] == 'Hipra Scientific S.L.U.')
]
choices = ['I', 'J', 'M']
v3['event'] = np.select(conditions, choices, default = 'NaN')
v3 = v3.rename({'v3_date':'date'}, axis = 1)

In [23]:
v3 = v3[['id', 'date', 'event']]
v3 = v3[v3['event'] != 'NaN']

In [24]:
v3['date'] = [str(datetime.strptime(i, '%d/%m/%Y').date()) for i in v3['date']]

In [25]:
# 4th dose
conditions = [
    (v4['v4'] == 'BioNTech / Pfizer'),
    (v4['v4'] == 'Moderna / Lonza')
]
choices = ['I', 'J']
v4['event'] = np.select(conditions, choices, default = 'NaN')
v4 = v4.rename({'v4_date':'date'}, axis = 1)

In [26]:
v4 = v4[['id', 'date', 'event']]
v4 = v4[v4['event'] != 'NaN']

In [27]:
v4['date'] = [str(datetime.strptime(i, '%d/%m/%Y').date()) for i in v4['date']]

In [29]:
# removing indetermined
sero = sero[sero['sero_result'] != 'I']

In [30]:
conditions = [
    (sero['sero_result'] == 'P'),
    (sero['sero_result'] == 'N')
]
choices = ['G', 'H']
sero['event'] = np.select(conditions, choices, default = 'NaN')
sero = sero.rename({'sero_date':'date'}, axis = 1)

In [31]:
sero = sero[['id', 'date', 'event']]
sero = sero[sero['event'] != 'NaN']

In [32]:
sero['date'] = [str(datetime.strptime(i, '%m/%d/%Y').date()) for i in sero['date']]

In [33]:
# PCR (also antigen)
pcr['test'].value_counts()

PCR        43345
Antigen    10011
Name: test, dtype: int64

In [34]:
# removing 'Reacció inhibida'
pcr = pcr[pcr['pcr_result'] != 'Reacció inhibida']

In [35]:
conditions = [
    (pcr['test'] == 'PCR') & (pcr['pcr_result'] == 'Positiu'),
    (pcr['test'] == 'PCR') & (pcr['pcr_result'] == 'Negatiu'),
    (pcr['test'] == 'Antigen') & (pcr['pcr_result'] == 'Positiu'),
    (pcr['test'] == 'Antigen') & (pcr['pcr_result'] == 'Negatiu')
]
choices = ['A', 'B', 'E', 'F']
pcr['event'] = np.select(conditions, choices, default = 'NaN')
pcr = pcr.rename({'pcr_date':'date'}, axis = 1)

In [36]:
pcr = pcr[['id', 'date', 'event']]
pcr = pcr[pcr['event'] != 'NaN']

In [37]:
pcr['date'] = [str(datetime.strptime(i, '%m/%d/%Y').date()) for i in pcr['date']]

In [38]:
# test 1 and test 2
# setting NaN cells in CT to 99 to avoid errors
test1['ct_1'] = test1['ct_1'].fillna(99)
test2['ct_2'] = test2['ct_2'].fillna(99)

In [39]:
# Positive PCR/TMA + information about the possibility to sequence the virus
# we will use petition date
conditions = [
    (test1['seq_1'] == 'X'),
    (test1['seq_1'] != 'X') & (test1['ct_1'] <= 33),
    (test1['seq_1'] != 'X') & (test1['ct_1'] > 33)
]
choices = ['C', 'D', 'A']
test1['event'] = np.select(conditions, choices, default = 'NaN')
test1 = test1.rename({'pet_date_1':'date'}, axis = 1)
test1 = test1[['id', 'date', 'event']]
test1 = test1[test1['event'] != 'NaN']

In [40]:
conditions = [
    (test2['seq_2'] == 'X'),
    (test2['seq_2'] != 'X') & (test2['ct_2'] <= 33),
    (test2['seq_2'] != 'X') & (test2['ct_2'] > 33)
]
choices = ['C', 'D', 'A']
test2['event'] = np.select(conditions, choices, default = 'NaN')
test2 = test2.rename({'pet_date_2':'date'}, axis = 1)
test2 = test2[['id', 'date', 'event']]
test2 = test2[test2['event'] != 'NaN']

In [41]:
# two nan in both test1 and test2 dates
test1 = test1.dropna()
test2 = test2.dropna()

In [42]:
test1['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in test1['date']]
test2['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in test2['date']]

In [527]:
# Symptomatology (yes/no)
# Removing NaN (no symp. / not recorded)
symp = symp.dropna()

In [528]:
# removing duplicates
symp = symp.drop_duplicates()

In [531]:
# adding code
symp['event'] = 'U'

In [533]:
# changing date format
symp = symp.rename({'symp_date':'date'}, axis = 1)
symp['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in symp['date']]

In [43]:
# Hospitalization info
hosp

,id,entry_date,discharge_date,diag,diag_desc,uci_e_date,uci_d_date,ucri_e_date,ucri_d_date
0,7a074a447dac01a5,26/04/2020,24/05/2020,J12.89,Altres tipus de pneumonia virica,NaN,NaN,NaN,NaN
1,0bd682e57a2ae797,26/03/2020,13/04/2020,J12.89,Altres tipus de pneumonia virica,NaN,NaN,NaN,NaN
2,5041208e7cfffad3,22/04/2020,29/04/2020,J12.89,Altres tipus de pneumonia virica,NaN,NaN,NaN,NaN
3,270bed643e3cff47,02/04/2020,29/05/2020,J12.89,Altres tipus de pneumonia virica,NaN,NaN,NaN,NaN
4,bf94f05aa89c3a04,25/03/2020,21/04/2020,J12.89,Altres tipus de pneumonia virica,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
265,7dbde5cfb4b271fd,24/12/2021,11/01/2022,U07.1,COVID-19,NaN,NaN,NaN,NaN
266,695941382f0cbf24,05/05/2020,13/05/2020,B34.2,Infeccio per coronavirus no especificada,NaN,NaN,NaN,NaN
267,e454ba1ee0e12683,21/02/2022,25/02/2022,O23.03,"Infeccions de ronyo en l'embaras, tercer trime...",NaN,NaN,NaN,NaN
268,0f78716194f8658e,25/04/2021,06/05/2021,U07.1,COVID-19,NaN,NaN,NaN,NaN


In [44]:
# we need to split this df using the different fields
# entry date + info about the cause
entry = hosp[['id', 'entry_date', 'diag']]
# discharge date
disch = hosp[['id', 'discharge_date']]
# UCI/UCRI entry
uci_e = hosp[['id', 'uci_e_date', 'ucri_e_date']]
# UCI/UCRI discharge
uci_d = hosp[['id', 'uci_d_date', 'ucri_d_date']]

In [45]:
# starting with the easiest, discharge
disch['event'] = 'Q'
disch = disch.rename({'discharge_date':'date'}, axis = 1)

In [46]:
disch['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in disch['date']]

In [47]:
# uci_e
# removing rows with more than 1 NaN (not uci neither ucri)
uci_e = uci_e.dropna(thresh = 2)
uci_e['event'] = 'R'
uci_e

,id,uci_e_date,ucri_e_date,event
16,d8a456be5dab264f,NaN,01/12/2020,R
34,63c70a70a470c823,31/03/2020,07/04/2020,R
47,d046a208c69109ab,NaN,13/03/2021,R
49,a7ced75a81d73d28,26/08/2021,NaN,R
54,00cad70e300b78c8,NaN,21/10/2020,R
58,6dd8420771a8c9e2,19/03/2020,NaN,R
68,be18d3bb7d2ac643,NaN,10/04/2020,R
71,85098acd9dcb74f1,NaN,02/12/2020,R
81,3870cc46777b8408,14/01/2021,NaN,R
108,9bffdae9a9ce89e3,NaN,14/05/2021,R


In [48]:
# some patients were entered in both UCI and UCRI
# we will keep the first date
records = uci_e.to_records(index=False)
result = list(records)

# we have to format dates (YYYY-MM-DD) to be able to operate with them
dates = []
for tup in result:
    if pd.isna(tup[1]):
        dates.append(str(datetime.strptime(tup[2], '%d/%m/%Y').date()))
    elif pd.isna(tup[2]):
        dates.append(str(datetime.strptime(tup[1], '%d/%m/%Y').date()))
    else:
        x = datetime.strptime(tup[1], '%d/%m/%Y')
        y = datetime.strptime(tup[2], '%d/%m/%Y')
        dates.append(str(min(x.date(), y.date())))
dates

# we will go back and change all dates in other tables to match this format

['2020-12-01',
 '2020-03-31',
 '2021-03-13',
 '2021-08-26',
 '2020-10-21',
 '2020-03-19',
 '2020-04-10',
 '2020-12-02',
 '2021-01-14',
 '2021-05-14',
 '2022-01-10',
 '2021-07-17',
 '2020-12-31',
 '2020-07-24',
 '2020-10-12',
 '2020-08-20',
 '2020-10-29',
 '2020-08-21',
 '2021-01-27',
 '2020-04-12',
 '2020-07-17',
 '2020-04-01',
 '2020-12-05',
 '2020-04-06',
 '2020-12-02',
 '2020-04-09',
 '2020-12-27',
 '2021-07-26',
 '2021-01-05',
 '2021-03-15',
 '2021-01-19',
 '2021-04-05',
 '2020-04-05',
 '2021-12-04']

In [49]:
uci_e['date'] = dates

In [50]:
uci_e = uci_e[['id', 'date', 'event']]

In [51]:
# uci_d
uci_d = uci_d.dropna(thresh = 2)
uci_d['event'] = 'S'

In [52]:
# same as uci_e
records = uci_d.to_records(index=False)
result = list(records)
dates = []
for tup in result:
    if pd.isna(tup[1]):
        dates.append(str(datetime.strptime(tup[2], '%d/%m/%Y').date()))
    elif pd.isna(tup[2]):
        dates.append(str(datetime.strptime(tup[1], '%d/%m/%Y').date()))
    else:
        x = datetime.strptime(tup[1], '%d/%m/%Y')
        y = datetime.strptime(tup[2], '%d/%m/%Y')
        dates.append(str(min(x.date(), y.date())))

In [53]:
uci_d['date'] = dates
uci_d = uci_d[['id', 'date', 'event']]

In [54]:
# for hospital entry we will use three codes
# N: probable COVID infection
# O: possible COVID infection
# P: ischemic embolic phenomena possibly associated with COVID
# these categories will be loaded into 'diag', and hospitalizations unmatching these cases will be ignored

# loading diagnosis sheet
diag = pd.read_csv('../data/Recovid_diagnosticos.csv', sep = '\t')

In [55]:
# removing unrelated to COVID entries
entry = entry[entry['diag'].isin(diag['DIAG_PPAL'])]

In [56]:
# getting lists for each category
c1 = diag[diag['Categoria'] == 1]
c2 = diag[diag['Categoria'] == 2]
c3 = diag[diag['Categoria'] == 3]

In [57]:
conditions = [
    (entry['diag'].isin(c1['DIAG_PPAL'])),
    (entry['diag'].isin(c2['DIAG_PPAL'])),
    (entry['diag'].isin(c3['DIAG_PPAL']))
]
choices = ['N', 'O', 'P']
entry['event'] = np.select(conditions, choices, default = 'NaN')
entry = entry.rename({'entry_date':'date'}, axis = 1)
entry = entry[['id', 'date', 'event']]
entry = entry[entry['event'] != 'NaN']

In [58]:
entry['event'].value_counts()

N    194
P      9
O      3
Name: event, dtype: int64

In [59]:
# formating date
entry['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in entry['date']]

In [334]:
# Decease
# Adding code
dec['event'] = 'T'

# Removing duplicates
dec = dec.drop_duplicates()

# Changing rownames
dec = dec.rename({'ID_ANONIMITZAT':'id', 'DATA_EXITUS':'date'}, axis = 1)

In [336]:
# Changing date format
dec['date'] = [str(datetime.strptime(str(i), '%d/%m/%Y').date()) for i in dec['date']]

In [158]:
# some tables have duplicated rows; removing them
test1 = test1.drop_duplicates()
test2 = test2.drop_duplicates()
sero = sero.drop_duplicates()
v1 = v1.drop_duplicates()
v2 = v2.drop_duplicates()
v3 = v3.drop_duplicates()

In [174]:
# Now we have all of our information with the same format
# we will join the dataframes
# most pcrs in test1 and test2 will also appear in pcr | we have to manage that
tests = pd.concat([test1, test2])

In [175]:
# sorting by id and date
tests = tests.sort_values(['id', 'date'], ascending = [True, True])

In [176]:
# resetting index
tests = tests.reset_index(drop = True)

In [177]:
# managing pcr dataframe
pcr = pcr.sort_values(['id', 'date'], ascending = [True, True])
pcr = pcr.reset_index(drop = True)

In [178]:
# merging dataframes to see if there are dup between tests and pcr
# if so we will keep data coming from tests as it contains info about seq
merged = pd.merge(pcr, tests, on = ['id', 'date'], how = 'outer', indicator = True)

In [179]:
merged 

,id,date,event_x,event_y,_merge
0,005cad4958846409,2020-08-14,B,NaN,left_only
1,005cad4958846409,2020-09-25,A,A,both
2,005cad4958846409,2020-11-04,A,NaN,left_only
3,005cad4958846409,2020-11-07,F,NaN,left_only
4,005cad4958846409,2020-11-17,A,NaN,left_only
...,...,...,...,...,...
53357,e2e9b909e1604997,2021-10-20,NaN,A,right_only
53358,e6f050fed0ca2b9e,2022-02-09,NaN,A,right_only
53359,eac6abc242cf5614,2021-02-26,NaN,A,right_only
53360,eac6abc242cf5614,2021-07-22,NaN,A,right_only


In [180]:
# keeping pcr-exclusive data
merged = merged[merged['_merge'] == 'left_only']
merged = merged[['id', 'date', 'event_x']]
merged = merged.rename({'event_x':'event'}, axis = 1)

In [182]:
# concatenating with tests
all_pcr = pd.concat([tests, merged])
all_pcr = all_pcr.sort_values(['id', 'date'], ascending = [True, True])
all_pcr = all_pcr.reset_index(drop = True)

In [183]:
# checking for duplicates
len(all_pcr[all_pcr.duplicated()])

0

In [184]:
# total number of PCR/TMA/antigen tests
len(all_pcr)

52784

In [535]:
# Concatenating all tables
ALL = pd.concat([all_pcr, v1, v2, v3, v4, sero, entry, disch, uci_e, uci_d, symp, dec])

In [536]:
ALL['event'].value_counts()

B    34519
F     8769
A     7753
I     4814
J     1642
G     1166
C      723
E      708
H      609
U      462
D      312
Q      270
N      194
T      121
K      109
L       95
R       34
S       34
P        9
O        3
M        1
Name: event, dtype: int64

In [537]:
# adding column for code explanation
conditions = [
    (ALL['event'] == 'A'),
    (ALL['event'] == 'B'),
    (ALL['event'] == 'C'),
    (ALL['event'] == 'D'),
    (ALL['event'] == 'E'),
    (ALL['event'] == 'F'),
    (ALL['event'] == 'G'),
    (ALL['event'] == 'H'),
    (ALL['event'] == 'I'),
    (ALL['event'] == 'J'),
    (ALL['event'] == 'K'),
    (ALL['event'] == 'L'),
    (ALL['event'] == 'M'),
    (ALL['event'] == 'N'),
    (ALL['event'] == 'O'),
    (ALL['event'] == 'P'),
    (ALL['event'] == 'Q'),
    (ALL['event'] == 'R'),
    (ALL['event'] == 'S'),
    (ALL['event'] == 'T'),
    (ALL['event'] == 'U')
]
choices = ['PCR/TMA+', 'PCR/TMA-', 'PCR, seq.', 'PCR, pot. seq.','Antigen+', 'Antigen-', 'Serology+', 'Serology-',
          'Pfizer', 'Moderna', 'AstraZeneca', 'Janssen','HIPRA', 'Hosp. entry (probable COVID-19)',
          'Hosp. entry (possible COVID-19)', 'Hosp. entry (ischemic embolic phenomena possibly derived from COVID-19)',
          'Hosp. discharge', 'UCI/UCRI entry', 'UCI/UCRI discharge', 'Decease', 'Symptomatology']
ALL['meaning'] = np.select(conditions, choices, default = 'NaN')

In [538]:
ALL['meaning'].value_counts()

PCR/TMA-                                                                   34519
Antigen-                                                                    8769
PCR/TMA+                                                                    7753
Pfizer                                                                      4814
Moderna                                                                     1642
Serology+                                                                   1166
PCR, seq.                                                                    723
Antigen+                                                                     708
Serology-                                                                    609
Symptomatology                                                               462
PCR, pot. seq.                                                               312
Hosp. discharge                                                              270
Hosp. entry (probable COVID-

In [539]:
# sorting and managing index
ALL = ALL.sort_values(['id', 'date'], ascending = [True, True])
ALL = ALL.reset_index(drop = True)

In [540]:
# example
ALL[ALL['id'] == '005cad4958846409']

,id,date,event,meaning
0,005cad4958846409,2020-08-14,B,PCR/TMA-
1,005cad4958846409,2020-09-25,A,PCR/TMA+
2,005cad4958846409,2020-11-04,A,PCR/TMA+
3,005cad4958846409,2020-11-06,H,Serology-
4,005cad4958846409,2020-11-07,F,Antigen-
5,005cad4958846409,2020-11-17,A,PCR/TMA+
6,005cad4958846409,2020-12-02,B,PCR/TMA-
7,005cad4958846409,2020-12-23,B,PCR/TMA-
8,005cad4958846409,2020-12-30,B,PCR/TMA-
9,005cad4958846409,2021-01-11,I,Pfizer


## 3. CODING

In [541]:
# ALL is a df with all events sorted by ID and date
# we need to transform this information in strings

# the first step is to convert dates in day differences
# for every ID, the first event will be set to 0 and successive differences will be calculated

# getting ID list
ids = list(ALL['id'].unique())

# adding a column in ALL to insert time differences
ALL['diff'] = 0

# converting 'date' column to datetime format to operate
ALL['date'] = [datetime.strptime(i, '%Y-%m-%d').date() for i in ALL['date']]

In [224]:
# for testing
# X = ALL.copy()

In [542]:
LIST = []
for x in ids:
    F = ALL[ALL['id'] == x]
    F = F.reset_index(drop = True)
    for i in range(1, len(F)):
        F.loc[i, 'diff'] = (F.iloc[i]['date'] - F.iloc[i-1]['date']).days
    LIST.append(F)

In [543]:
ALL = pd.concat([x for x in LIST])

In [544]:
ALL = ALL.reset_index(drop = True)

In [545]:
ALL

,id,date,event,meaning,diff
0,005cad4958846409,2020-08-14,B,PCR/TMA-,0
1,005cad4958846409,2020-09-25,A,PCR/TMA+,42
2,005cad4958846409,2020-11-04,A,PCR/TMA+,40
3,005cad4958846409,2020-11-06,H,Serology-,2
4,005cad4958846409,2020-11-07,F,Antigen-,1
...,...,...,...,...,...
62342,ffc2d5ab7d2ad39e,2020-07-16,B,PCR/TMA-,0
62343,ffc2d5ab7d2ad39e,2020-10-17,A,PCR/TMA+,93
62344,ffc2d5ab7d2ad39e,2020-10-19,U,Symptomatology,2
62345,ffc2d5ab7d2ad39e,2021-07-17,D,"PCR, pot. seq.",271


In [546]:
str(ALL.iloc[0]['event'] + str(ALL.iloc[1]['diff']) + ALL.iloc[1]['event'])

'B42A'

In [547]:
LIST[0]

,id,date,event,meaning,diff
0,005cad4958846409,2020-08-14,B,PCR/TMA-,0
1,005cad4958846409,2020-09-25,A,PCR/TMA+,42
2,005cad4958846409,2020-11-04,A,PCR/TMA+,40
3,005cad4958846409,2020-11-06,H,Serology-,2
4,005cad4958846409,2020-11-07,F,Antigen-,1
5,005cad4958846409,2020-11-17,A,PCR/TMA+,10
6,005cad4958846409,2020-12-02,B,PCR/TMA-,15
7,005cad4958846409,2020-12-23,B,PCR/TMA-,21
8,005cad4958846409,2020-12-30,B,PCR/TMA-,7
9,005cad4958846409,2021-01-11,I,Pfizer,12


In [548]:
d = {}
for i in range(len(ids)):
    string = LIST[i].iloc[0]['event']
    for j in range(1,len(LIST[i])):
        string = str(string + str(LIST[i].iloc[j]['diff']) + LIST[i].iloc[j]['event'])
    d[ids[i]] = [str(LIST[i].iloc[0]['date']), string]

In [549]:
d

{'005cad4958846409': ['2020-08-14',
  'B42A40A2H1F10A15B21B7B12I24I7F6B28B128A4D77B64J20B14B15F6F5F16F7F7F'],
 '0065b57e5a9e784f': ['2020-03-26',
  'A37B234F4B4B8B2F2B2F3B4F0G3B2I8G17I215I82B9B23A'],
 '006e5a1bf72e8b73': ['2020-06-02',
  'B56B7B35B11B3B11B10B13B14B8A178I21I123B6B5B6B5B6B5A24J'],
 '009d11a2497f1d47': ['2020-10-14', 'A334C142J'],
 '00a5cf0620b6adfc': ['2020-10-26', 'A201J213A'],
 '00cad70e300b78c8': ['2020-10-10', 'A9A1N1R1S11N0Q7A2G8Q301I132A34J6F'],
 '00f31c11a3a6f08d': ['2020-08-03', 'A143A111I21I188I'],
 '011e826ab054591f': ['2020-11-10', 'A176A29G88I140F'],
 '012a57320c31a43f': ['2020-08-07',
  'B84B15B17A13B7F1B6B2F5F0B5B2F5B2F4F0B3F2F2B3F2F1B4F2F2F1B2F12B14B4I10B13I1B14B14B53B30B31B5B7B7B7B14B7B14B14B14B14B2J7F4B11B3F1B1F6F0B3F4B2F3F2A1E'],
 '0134889c8d47909e': ['2020-03-16', 'A183A134B357E0U'],
 '01394866d1e64c72': ['2020-05-05',
  'B203B8B37I2A3B16I13B2F3B7B161B9B22B3B4B12I64B4B8A3B'],
 '0139873673801ea0': ['2020-04-05', 'A145B46B10B75F15A98B94F147F3F'],
 '01422

In [556]:
import random
r = random.randint(0,len(ids))
ALL[ALL['id'] == ids[r]]

,id,date,event,meaning,diff
47095,c06b4723b028c65c,2020-06-09,A,PCR/TMA+,0
47096,c06b4723b028c65c,2020-11-14,B,PCR/TMA-,158
47097,c06b4723b028c65c,2020-12-24,F,Antigen-,40
47098,c06b4723b028c65c,2020-12-26,B,PCR/TMA-,2
47099,c06b4723b028c65c,2021-01-02,B,PCR/TMA-,7
47100,c06b4723b028c65c,2021-01-05,F,Antigen-,3
47101,c06b4723b028c65c,2021-01-09,B,PCR/TMA-,4
47102,c06b4723b028c65c,2021-01-13,I,Pfizer,4
47103,c06b4723b028c65c,2021-01-23,B,PCR/TMA-,10
47104,c06b4723b028c65c,2021-01-30,B,PCR/TMA-,7


In [423]:
deceased = list(ALL[ALL['event'] == 'T']['id'])

In [505]:
counter = 0

In [513]:
print(ALL[ALL['id'] == deceased[counter]])
print(info[info['id'] == deceased[counter]])
counter = counter + 1
print(counter)

                     id        date event   meaning  diff
27692  748acfc89fcfd45f  2020-04-23     B  PCR/TMA-     0
27693  748acfc89fcfd45f  2020-10-08     A  PCR/TMA+   168
27694  748acfc89fcfd45f  2020-11-06     F  Antigen-    29
27695  748acfc89fcfd45f  2020-11-10     F  Antigen-     4
27696  748acfc89fcfd45f  2021-01-10     A  PCR/TMA+    61
27697  748acfc89fcfd45f  2021-01-13     B  PCR/TMA-     3
27698  748acfc89fcfd45f  2021-01-15     I    Pfizer     2
27699  748acfc89fcfd45f  2021-02-08     I    Pfizer    24
27700  748acfc89fcfd45f  2021-02-19     B  PCR/TMA-    11
27701  748acfc89fcfd45f  2021-02-22     F  Antigen-     3
27702  748acfc89fcfd45f  2021-02-24     B  PCR/TMA-     2
27703  748acfc89fcfd45f  2021-03-03     B  PCR/TMA-     7
27704  748acfc89fcfd45f  2021-09-19     I    Pfizer   200
27705  748acfc89fcfd45f  2021-12-28     B  PCR/TMA-   100
27706  748acfc89fcfd45f  2022-01-19     T   Decease    22
                   id hosp   age   sex maca  pcc    gma   symp_date
679 

In [518]:
info['gma'].value_counts()

331.0    420
322.0    351
332.0    336
321.0    301
333.0    227
334.0    224
1.0      186
323.0    169
312.0    158
101.0    126
311.0    126
313.0    115
324.0     84
314.0     83
102.0     66
335.0     41
103.0     38
315.0     32
325.0     30
202.0     28
201.0     22
401.0     19
104.0     14
403.0     12
105.0     12
402.0     12
203.0      9
204.0      6
404.0      4
205.0      1
Name: gma, dtype: int64

In [557]:
# saving results
csv = []
for k,v in d.items():
    y = []
    y.append(k)
    y.append(v[0])
    y.append(v[1])
    csv.append(y)

In [558]:
CSV = pd.DataFrame(csv, columns = ['id', 'first_event_date', 'code'])
CSV.to_csv('../patients_w_codes.csv', index = False)

In [559]:
# saving ALL dataframe as well
ALL.to_csv('../patients_w_timeline.csv', index = False)

In [419]:
# saving info df for further characterization
info = info.drop_duplicates()
info.to_csv('../data/info.tsv', sep = '\t', index = False)

In [ ]:
# ANOTHER VERSION; CODING FIRST EVENT DATE INTO STRING
D = {}
for i in range(len(ids)):
    string = LIST[i].iloc[0]['event']
    string = '[' + str(LIST[i].iloc[0]['date']) + ']' + string
    for j in range(1, len(LIST[i])):
        string = str(string + str(LIST[i].iloc[j]['diff']) + LIST[i].iloc[j]['event'])
    D[ids[i]] = string

In [ ]:
csv2 = []
for k,v in D.items():
    z = []
    z.append(k)
    z.append(v)
    csv2.append(z)

In [ ]:
CSV2 = pd.DataFrame(csv2, columns = ['id', 'code'])
CSV2.to_csv('../results/patients_w_codes2.csv', index = False)